In [ ]:
import numpy as np
import pandas as pd
import re
import torch
import shutil
import html
import string
!pip install demoji
import demoji
import torch.nn as nn
from nltk.stem.wordnet import WordNetLemmatizer
import csv
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 KB 308.9 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/sydney-machine-learning/COVID19_sentinentanalysis

Cloning into 'COVID19_sentinentanalysis'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 309 (delta 61), reused 134 (delta 52), pack-reused 165
Receiving objects: 100% (309/309), 93.07 MiB | 17.45 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
!pip install transformers==4.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=140049e063814bdea6f8c9128e30af3d8238ea6c820233231ed7bc42c6204be7
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


**Australia**

In [ ]:
aus = pd.read_csv("/content/drive/MyDrive/Tweets_Australia/Tweets_Australia.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
aus = aus.drop(columns=['Unnamed: 0', 'created_at', 'id', 'user_location'])
aus.head()

,text
0,RT @AaronSiriSG: Litigation update: \n\nFDA d...
1,RT @covidbaseau: The TGA has approved the Mode...
2,RT @SimoLove: Federal Health Minister Greg Hun...
3,@twityouse Are these the deniers that are wear...
4,RT @drsimonegold: WOW: a preprint study for Th...


In [ ]:
aus.isnull().sum()

text    7
dtype: int64

In [ ]:
aus = aus.dropna(axis=0)

In [ ]:
aus.isnull().sum()

text    0
dtype: int64

In [ ]:
def vax(df):
  
  antivaxxer = df[df['text'].str.find('antivaxxer') != -1]
  antivaxxer = antivaxxer.reset_index()

  vaxxer = df[df['text'].str.find('vaxxer') != -1]
  vaxxer = vaxxer.reset_index()
  final = antivaxxer.append(vaxxer)

  vaccine = df[df['text'].str.find('vaccine') != -1]
  vaccine = vaccine.reset_index()
  final= final.append(vaccine)

  pfizer = df[df['text'].str.find('pfizer') != -1]
  pfizer = pfizer.reset_index()
  final = final.append(pfizer)

  BioNTech = df[df['text'].str.find('biontech') != -1]
  BioNTech = BioNTech.reset_index()
  final = final.append(BioNTech)

  sinopharm = df[df['text'].str.find('sinopharm') != -1]
  sinopharm = sinopharm.reset_index()
  final = final.append(sinopharm)

  sinovac = df[df['text'].str.find('sinovac') != -1]
  sinovac = sinovac.reset_index()
  final = final.append(sinovac)

  moderna = df[df['text'].str.find('moderna') != -1]
  moderna = moderna.reset_index()
  final = final.append(moderna)

  astra = df[df['text'].str.find('astrazeneca') != -1]
  astra = astra.reset_index()
  final = final.append(astra)

  covaxin = df[df['text'].str.find('covaxin') != -1]
  covaxin = covaxin.reset_index()
  final = final.append(covaxin)

  covi = df[df['text'].str.find('covishield') != -1]
  covi = covi.reset_index()
  final = final.append(covi)

  sput = df[df['text'].str.find('sputnik v') != -1]
  sput = sput.reset_index()
  final = final.append(sput)

  jan = df[df['text'].str.find('janssen') != -1]
  jan = jan.reset_index()
  final = final.append(jan)

  coro = df[df['text'].str.find('coronavac') != -1]
  coro = coro.reset_index()
  final = final.append(coro)

  nova = df[df['text'].str.find('novavax') != -1]
  nova = nova.reset_index()
  final = final.append(nova)

  convi = df[df['text'].str.find('convidecia') != -1]
  convi = convi.reset_index()
  final = final.append(convi)

  sanofi = df[df['text'].str.find('sanofi') != -1]
  sanofi = sanofi.reset_index()
  final = final.append(sanofi)

  return final 

In [ ]:
data = vax(aus)

In [ ]:
data = data.drop(columns=['index'],axis=1)

In [ ]:
data = data.reset_index()

In [ ]:
data = data.drop(columns=['index'],axis=1)

In [ ]:
data

,text
0,@Gr3yJ3d1 @420_BTC @newscomauHQ I just can’t i...
1,RT @krunchymoses: Oh wow. So antivaxxers in Me...
2,I think boosters at 3 months might be better f...
3,#PSA: This is your regular reminder that anyth...
4,RT @MatthewSloly: Canada’s antimaskers/antivax...
...,...
516554,RT @sanofi: This clinical trial will assess sa...
516555,"RT @coinspeaker: SAN Stock Up 0.52%, Sanofi to..."
516556,RT @sanofipasteur: Today we take the first ste...
516557,"RT @sanofi: In partnership with @TranslateBio,..."


In [ ]:
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [ ]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [ ]:
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions
    
    def lower(self, tweet):
        return tweet.lower()
    
    def abbreviate(self, tweet):
        tweet = tweet.split(' ')
        j = 0
        for str_ in tweet:
            fileName = '/content/COVID19_sentinentanalysis/preprocessing/Abbreviations.txt'
            accessMode = 'r'
            with open(fileName, accessMode) as csvfile:
                dataFromFile = csv.reader(csvfile, delimiter = '=')
                str_ = re.sub('[^a-zA-Z0-9-_.]', '', str_)
                for row in dataFromFile:
                    if str_.upper() == row[0]:
                        tweet[j] = row[1]
                csvfile.close()
            j += 1
        return ' '.join(tweet)
    
    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet
    
    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])
        
        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)
    
    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def remove_url(self, tweet):
        return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags = re.MULTILINE)    
    
    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_url(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)
        
        return tweet

In [ ]:
pp_class = preprocess(data, contractions, contractionsWithAnotherInvertedComma)
data['text'] = data['text'].apply(lambda x : pp_class.preprocess_tweet(x))

In [ ]:
data['text']

0            i just cannot imagine selfish antivaxxers r...
1         rt  oh wow  so antivaxxers in melbourne are pr...
2         i think boosters at 3 months might be better f...
3          psa  this is your regular reminder that anyth...
4         rt  canada’s antimaskers antivaxxers are not a...
                                ...                        
516554    rt  this clinical trial will assess safety  im...
516555    rt  san stock up 0 52   sanofi to begin human ...
516556    rt  today we take the first steps towards deve...
516557    rt  in partnership with  today we announce pre...
516558    rt  covid 19 pushed the 2 global leaders of va...
Name: text, Length: 516559, dtype: object

In [ ]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking']

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
df = pd.DataFrame()
df['Tweet'] = data['text']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * 516559
df[labels] = values

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)

   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }  

In [ ]:
MAX_LEN=256

In [ ]:
aus_dataset = CustomDataset(df, tokenizer, MAX_LEN)

In [ ]:
aus_data_loader = torch.utils.data.DataLoader(
    aus_dataset,
    shuffle=False,
    batch_size=8,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 11)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final

model = BERT()
model.to(device)

In [ ]:
model = torch.load("/content/drive/MyDrive/bertmodel.pth")

In [ ]:
model.eval()
bert_outputs = []
with torch.no_grad():

  for index, batch in enumerate(aus_data_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

In [ ]:
test_outputs = np.array(bert_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [ ]:
df[labels] = test_outputs.astype(int)

In [ ]:
df.head()

In [ ]:
df.to_csv('/content/drive/MyDrive/faus.csv')